In [ ]:
import matplotlib.pyplot as plt
import scipy
import scipy.signal
import numpy as np
import pandas as pd
import plotly.express as px
import skimage

file = r"J:\ctgroup\Edward\DATA\VMI\20240806\calibrated\xe_03_Scan5W_calibrated.h5"
data = pd.read_hdf(file, key="data")
inverse_data = data.copy()
inverse_data['px'] = -data['px']
inverse_data['pz'] = -data['pz']
inverse_data['py'] = -data['py']
data = pd.concat([data, inverse_data], ignore_index=True)

In [ ]:

data = data.query("-1<=px<=1 and -1<=py<=1 and -1<=pz<=1")
data['pr'] = np.sqrt(data['px'] ** 2 + data['py'] ** 2 + data['pz'] ** 2)
data['inv_pr'] = 1 / data['pr']
data['theta'] = np.arccos(data['py'] / data['pr'])
data['phi'] = np.arctan2(data['px'], data['pz'])

data_f = data.query('-0.35<=pr<=0.35 and -0.05<=py<=0.05')

hist, xe, ye = np.histogram2d(
        data_f['px'], data_f['pz'],
        bins=512,
)

xe = 1 / 2 * (xe[1:] + xe[:-1])
ye = 1 / 2 * (ye[1:] + ye[:-1])
signal = skimage.filters.difference_of_gaussians(
        hist.T,
        low_sigma=1,
        high_sigma=100,
)

px.imshow(
        signal,
        x=xe, y=ye,
        aspect='equal',
        height=1200, width=1200,
).show()

polar_signal = skimage.transform.warp_polar(signal, output_shape=(1024, 1024), radius=256).T
polar_signal = np.hstack((polar_signal, polar_signal, polar_signal))

# polar_signal=skimage.filters.difference_of_gaussians(
#         polar_signal,
#         low_sigma=5,
#         high_sigma=15,
# )

polar_min = scipy.signal.argrelmin(polar_signal, axis=1)
fig = px.imshow(
        polar_signal,
        height=1200, width=1200,
        origin='lower'
)

fig.add_scatter(x=polar_min[1], y=polar_min[0], mode='markers', marker=dict(size=5, color='red'))

fig.show()


In [ ]:
def fit_fn(theta, theta0, a, b):
    return a * np.cos((theta - theta0) * 4 * np.pi / 1024) + b


fit_params = []
for r in range(polar_signal.shape[0]):
    p0 = [0, np.max(polar_signal[r]) - np.min(polar_signal[r]), np.mean(polar_signal[r])]
    fit_params.append(scipy.optimize.curve_fit(
            fit_fn,
            xdata=np.arange(polar_signal.shape[1]),
            ydata=polar_signal[r],
            p0=p0,
    )[0])
fit_params = np.array(fit_params)
fig = px.imshow(
        polar_signal[:, :1024],
        x=np.arange(polar_signal.shape[1] // 3), y=np.arange(polar_signal.shape[0]),
        aspect='equal',
        height=600, width=1200,
        origin='lower',

)
theta0 = np.unwrap(fit_params[:, 0], period=128 / 4)
modulation_scale = fit_params[:, 1]
fig.add_scatter(x=theta0 + 1024, y=np.arange(polar_signal.shape[0]), mode='markers', marker=dict(size=5, color='red'))
fig.add_scatter(x=polar_min[1][polar_min[1] < 1024], y=polar_min[0][polar_min[1] < 1024], mode='markers',
                marker=dict(size=5, color='black'))

In [ ]:
r_vec = np.linspace(0, 0.35, 1024)
theta_vec = theta0 * 4 * np.pi / 1024
e_vec = r_vec ** 2 / 2
dt_de = np.gradient(theta_vec, e_vec)
total_signal = np.histogram(data['pr'], bins=1024, range=(0, 0.35))[0]

df = pd.DataFrame({'r': r_vec, 'theta': -theta_vec, 'E': e_vec, 'dtheta/dE': -dt_de, "Group Delay": -2 * dt_de,
                   "Total Signal": total_signal, "Modulation": abs(modulation_scale)})
df = df.query("0.007<E<0.036")

mask = (0.007 < e_vec) & (e_vec < 0.036)

r_vec, theta_vec, e_vec, dt_de = r_vec[mask], theta_vec[mask], e_vec[mask], dt_de[mask]

fig = px.imshow(
        hist.T,
        x=xe, y=ye,
        aspect='equal',
        height=1200, width=1200,
)

fig.add_scatter(x=r_vec * np.cos(theta_vec), y=r_vec * np.sin(theta_vec))
fig.show()

px.line(x=e_vec, y=-theta_vec).show()

fig = px.line(
        df,
        x='E', y='Group Delay',
        height=600, width=1200,
)
fig.add_hline(y=142, annotation={"text": "1 Period"})
fig.add_hline(y=284, annotation={"text": "2 Periods"})
fig.add_hline(y=426, annotation={"text": "3 Periods"})
fig.show()
px.line(df, x='E', y='Modulation', height=300, width=1200).show()
df_cos_fit = df.copy()

In [ ]:
sf = signal
sf -= np.min(sf)
sf = sf / np.max(sf)
sf = sf ** (1 / 2)
px.imshow(sf, x=xe, y=ye, aspect='equal', height=1200, width=1200).show()

sf = skimage.filters.meijering(sf)
# sf= sf>0.1
# sf= skimage.morphology.isotropic_erosion(sf, radius=5)
px.imshow(sf, x=xe, y=ye, aspect='equal', height=1200, width=1200).show()

sf = skimage.morphology.skeletonize(sf > 0.15)

pt_idx = np.argwhere(sf)
pts = xe[pt_idx[:, 1]], ye[pt_idx[:, 0]]

fig = px.imshow(
        hist.T,
        x=xe, y=ye,
        aspect='equal',
        height=1200, width=1200,
)
fig.add_scatter(x=pts[0], y=pts[1], mode='markers', marker=dict(size=5, color='red'))
fig.show()

from sklearn.cluster import DBSCAN

db = DBSCAN(eps=0.01, min_samples=5).fit(np.array(pts).T)
labels = db.labels_

px.scatter(x=np.arctan2(pts[1], pts[0]), y=np.sqrt(pts[0] ** 2 + pts[1] ** 2), color=labels).show()

In [ ]:
main_curve = pts[0][labels == 3], pts[1][labels == 3]
df = pd.DataFrame({'x': main_curve[0], 'y': main_curve[1]})
df['r'] = np.sqrt(df['x'] ** 2 + df['y'] ** 2)
df.sort_values(by='r', inplace=True)
df['theta'] = -np.arctan2(df['y'], df['x']) - np.pi
df['theta'] = np.unwrap(df['theta'])
df.sort_values(by='theta', inplace=True)
fig = px.scatter(x=df['theta'], y=df['r'])
spline = scipy.interpolate.make_smoothing_spline(df['theta'], df['r'], lam=0.5)
fig.add_scatter(x=df['theta'], y=spline(df['theta']), mode='lines')
fig.show()
df['r_spline'] = spline(df['theta'])
df['E'] = df['r_spline'] ** 2 / 2
df['dtheta/dE'] = df['theta'] / df['E']
df['Group Delay'] = 2 * df['dtheta/dE']
px.line(df, x='E', y='Group Delay', height=600, width=1200).add_hline(y=142, annotation={"text": "1 Period"}).add_hline(
    y=284, annotation={"text": "2 Periods"}).show()

In [ ]:
(
    px.imshow(
            hist.T,
            x=xe, y=ye,
            aspect='equal',
            height=800, width=800,
            origin='lower',
    ).add_scatter(
            x=df['r_spline'] * np.cos(df['theta']), y=df['r_spline'] * np.sin(-df['theta']),
            mode='lines',
    ).add_scatter(
            x=df_cos_fit['r'] * np.cos(-df_cos_fit['theta']), y=df_cos_fit['r'] * np.sin(-df_cos_fit['theta']),
            mode='lines',
    ).show()
)